# Sensitivity analysis of previous work for all experiments and orignal models

Allow editing using PyCharm (auto reloading)

In [0]:
%load_ext autoreload
%autoreload 2

Needed for compatibility when using both CoLab and Local Jupyter notebook. It sets the appropriate file path for the data and also installs local packages such as models and data_loading.

In [2]:
import os
import importlib

if os.getcwd() == '/content':
    from google.colab import drive
    drive.mount('/content/gdrive')
    FILE_PATH = '/content/gdrive/My Drive/Level-4-Project/data/'
    !cd gdrive/My\ Drive/Level-4-Project/ && pip install --editable .
    os.chdir('gdrive/My Drive/Level-4-Project/')
    
else:
    FILE_PATH = "C:/Users/macka/Google Drive/Level-4-Project/data/"
    
from src.models.original_models import cifar_initialised, cifar_xdeg, mixed, mixed_angle_A, non_initialised_X, xs
from src.data import load_data

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Obtaining file:///content/gdrive/My%20Drive/Level-4-Project
  Found existing installation: src 0.1.0
    Can't uninstall 'src'. No files were found to uninstall.
  Running setup.py develop for src


Using TensorFlow backend.


Import remaining packages

In [0]:
import numpy as np
import sys
from six.moves import cPickle
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils import np_utils
import sys
from sklearn.metrics import classification_report, confusion_matrix
import csv
from keras.models import load_model

Needed as originally code was for theano backend but now using tensor flow

In [0]:
from keras import backend as K
K.set_image_dim_ordering('th')

In [0]:
experiment_names = ["cifar_initialised", "cifar_xdeg", "mixed", "mixed_angle_A", "non_initialised_X", "xs"]
experiments = {}
for name in experiment_names:
    experiments[name] = {
        "models": [],
        "data_folders":[],
        "extra_params":{
            "nb_classes": None,
            "nb_train_samples": None
        }}

In [0]:
img_rows, img_cols = 75, 75

TODO:
"cifar_initialised",   [*]
"cifar_xdeg", [*]
"mixed",
"mixed_angle_A", [*]
"non_initialised_X", [*]
"xs" [*]

In [0]:
nb_classes = 7
for version_number in range(1,5):
    experiments["cifar_initialised"]["models"].append(
        cifar_initialised.make_model(version_number, img_rows, img_cols, nb_classes))
experiments["cifar_initialised"]["data_folders"].append("cifar_initialised")
experiments["cifar_initialised"]["extra_params"]["nb_classes"] = nb_classes
experiments["cifar_initialised"]["extra_params"]["nb_train_samples"] = 35595

In [0]:
nb_classes = 6
experiments["cifar_xdeg"]["models"].append(
    cifar_xdeg.make_model(img_rows, img_cols, nb_classes))
experiments["cifar_xdeg"]["data_folders"] = ['cifar_30deg', 'cifar_45deg', 'cifar_60deg']
experiments["cifar_xdeg"]["extra_params"]["nb_classes"] = nb_classes
experiments["cifar_xdeg"]["extra_params"]["nb_train_samples"] = 34260

In [0]:
# ADD EARLY STOPPING TO MIXED? (Takes the longest by a significant amount)
nb_classes = 7
for version_number in range(1,3):
    experiments["mixed"]["models"].append(
        mixed.make_model(version_number, img_rows, img_cols, nb_classes))
experiments["mixed"]["data_folders"].append("mixed")
experiments["mixed"]["extra_params"]["nb_classes"] = nb_classes
experiments["mixed"]["extra_params"]["nb_train_samples"] = 52695

In [0]:
nb_classes = 7
experiments["mixed_angle_A"]["models"].append(
    mixed_angle_A.make_model(img_rows, img_cols, nb_classes))
experiments["mixed_angle_A"]["data_folders"].append("mixed_angle_A")
experiments["mixed_angle_A"]["extra_params"]["nb_classes"] = nb_classes
experiments["mixed_angle_A"]["extra_params"]["nb_train_samples"] = 43450

In [0]:
nb_classes = 7
experiments["non_initialised_X"]["models"].append(
    non_initialised_X.make_model(img_rows, img_cols, nb_classes))
experiments["non_initialised_X"]["data_folders"] = [
    'noninitialised_A',
    'noninitialised_B',
    'noninitialised_C',
    'noninitialised_D']
experiments["non_initialised_X"]["extra_params"]["nb_classes"] = nb_classes
experiments["non_initialised_X"]["extra_params"]["nb_train_samples"] = 34720

In [0]:
nb_classes = 7
experiments["xs"]["models"].append(xs.make_model(img_rows, img_cols, nb_classes))
experiments["xs"]["data_folders"] = ['1s', '1_5s', '2s']
experiments["xs"]["extra_params"]["nb_classes"] = nb_classes
experiments["xs"]["extra_params"]["nb_train_samples"] = 34720

In [0]:
batch_size = 100
nb_epoch = 1
data_augmentation = False

In [14]:
for name, experiment in experiments.items():
    for model in experiment["models"]:
        for data_folder in experiment["data_folders"]:
            print(model, data_folder)
            # the data, shuffled and split between train and test sets
            path = FILE_PATH + data_folder
            nb_train_samples = experiment["extra_params"]["nb_train_samples"]
            nb_classes = experiment["extra_params"]["nb_classes"]
            (X_train, y_train), (X_test, y_test) = load_data.load_data(path, nb_train_samples)
            print('X_train shape:', X_train.shape)
            print(X_train.shape[0], 'train samples')
            print(X_test.shape[0], 'test samples')

            # convert class vectors to binary class matrices
            Y_train = np_utils.to_categorical(y_train, nb_classes)
            Y_test = np_utils.to_categorical(y_test, nb_classes)
            
            # let's train the model using SGD + momentum (how original).
            sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
            model.compile(loss='categorical_crossentropy',
                          optimizer=sgd,
                          metrics=['accuracy'])

            X_train = X_train.astype('float32')
            X_test = X_test.astype('float32')
            X_train /= 255
            X_test /= 255 
            if not data_augmentation:
                print('Not using data augmentation.')
                hist = model.fit(X_train, Y_train,
                          batch_size=batch_size,
                          epochs=nb_epoch,
                          validation_data=(X_test, Y_test),
                          shuffle=True)

#                 y_pred = model.predict_classes(X_test)
#                 print(y_pred)
#                 target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting', 'Walking']
#         #         sys.stdout = open('/home/aleksandar/sets/cifar_initialised/results/report_initialised.txt', "w")
#                 print("CLASSIFICATION REPORT:")
#                 print(classification_report(np.argmax(Y_test,axis=1), y_pred,target_names=target_names))
#                 print ("\n")
#                 print("CONFUSION MATRIX:")
#                 print(confusion_matrix(np.argmax(Y_test,axis=1), y_pred))

<keras.models.Sequential object at 0x7faeae6037f0> cifar_initialised
X_train shape: (35595, 1, 75, 75)
35595 train samples
7119 test samples
Not using data augmentation.
Train on 35595 samples, validate on 7119 samples
Epoch 1/1
35595/35595 [==============================] - 21s 582us/step - loss: 0.5588 - acc: 0.7834 - val_loss: 0.0799 - val_acc: 0.9782
<keras.models.Sequential object at 0x7faea8cb3908> cifar_initialised
X_train shape: (35595, 1, 75, 75)
35595 train samples
7119 test samples
Not using data augmentation.
Train on 35595 samples, validate on 7119 samples
Epoch 1/1
35595/35595 [==============================] - 32s 892us/step - loss: 0.6319 - acc: 0.7575 - val_loss: 0.0889 - val_acc: 0.9659
<keras.models.Sequential object at 0x7faea8b7f630> cifar_initialised
X_train shape: (35595, 1, 75, 75)
35595 train samples
7119 test samples
Not using data augmentation.
Train on 35595 samples, validate on 7119 samples
Epoch 1/1
35595/35595 [==============================] - 8s 214us/s